<a href="https://colab.research.google.com/github/rinikhaneja/TECH16-LLM-for-business-with-Python/blob/main/Smart_Career_Coach_Agent_TECH16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Smart Career Coach Agent** is an intelligent, feedback-driven career guidance system designed to help job seekers prepare effectively for technical interviews by refining their resumes, upgrading their skills, and aligning with current market demands.

**Target Users:**
Job seekers preparing for interviews who want focused, real-world guidance instead of generic career advice.

**How It Works:** (4 Agents used with Sequential and Hierarchical Process)
1.Resume Analyzer Agent - Summarizes experience, identifies key skills and outdated tech.
2.Job Search Agent - Fetches relevant jobs using SerperAPI.
3.Interview Agent - Interview Questions in market for the job role.
4.Career Critic Agent - Gives career improvement tips, resume rewrites.

**Future Plan (To Turn into a Profitable Business):**

1.Create a Simple Web UI - User-friendly webpage where users can upload their resumes and instantly receive career analysis, job matches, and feedback reports.

2.Integrate with My Job Search Project- Link the existing project that filters job listings with at least a 75% match to the user's resume. This will ensure better personalization and relevance.

3.API Integrations for Richer Recommendations -
  a. Job Search APIs: Connect with LinkedIn, Wellfound, and Google Jobs to fetch more relevant job listings.
  b. Learning Paths: Recommend upskilling resources using platforms like Coursera, DeepLearning.ai, and Pluralsight.
  c. Apply Assistance: Support "Easy Apply" through integration with LinkedIn and other ATS platforms.

4.Monetization Strategy
B2C:
Free tier: 1-time resume analysis and job suggestions
Paid plans: Detailed coaching, job match insights, interview prep, resume rewriting

B2B:
Offer to universities, bootcamps, and HR teams as a career readiness tool or placement support service



     
   

In [ ]:
!pip install "crewai[tools]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 68.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 650.1/650.1 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.5/119.5 kB 10.1 MB/s 

In [ ]:
import os
import openai
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')


Sequential Agents

In [ ]:
!pip install langchain-openai
from types import resolve_bases
# Import CrewAI components and tools
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool, WebsiteSearchTool
from IPython.display import Markdown, display


#---- User input/Resume text------
job_title = "Software Engineer"
yrs_of_exp = "5"
skills = ["Java", "Python", "Spring-Boot", "Spring MVC", "Oracle",
        "Cassandra", "Couchbase", "GIT", "Grafana", "Prometheus",
        "Kafka", "Microsoft Azure", "Java Script"]
work_exp=["Secured Kafka data streams using OAuth 2.0 authentication integrated with MicrosoE Azure.",
          "Adding application security for secure communication of applications.",
          "Added 3 APIs with CRUD feature for a customer",
          "Worked on JIRA stories"]
city_location = "SFO"

# ---Format skills and work exp for inclusion in goal strings-----
formatted_skills = ", ".join(skills)  # e.g., "Java, Python, Spring-Boot, ..."
formatted_work_exp = "\n".join(work_exp)

#---- Enables web searching via the Serper API AND Retrieves and reads content from web pages----
serper_api_dev_tool = SerperDevTool()
web_reader_tool = WebsiteSearchTool()  # Retrieves and reads content from web pages

#----Create all AGENTS-----------
# Resume Analyser agent goal is to analyse resume extract key skills, exp summary, and outdated tech
resume_analyzer_agent = Agent(
    name="Analyser",
    role="Resume Analyser",
    goal=("Extract job title '{job_title}', '{yrs_of_exp}' years of experience, skills '{formatted_skills}' and work experience '{formatted_work_exp}' "
    "Identify the job title, outdated or weak skills from the list of extracted skills, provided in the resume text."
    "Do not give text which is not present in the '{formatted_skills}' and '{formatted_work_exp}'."
    "Summarize it in a short paragraph."
    ),
    backstory=("A senior career analyst specializing in resume mining. With years of expertise in technical hiring, the agent detects strengths, blind spots, "
        "and recommends learning areas based on skill trends."
    ),
    tools=[] #not required for this agent
    )

# Job Search agent retrieves relevant jobs from  SerperAPI
job_search_agent = Agent(
    name="Job Finder",
    role="Market Research and Job Search Agent",
    goal=("Search and retrieve the most relevant and high-quality job listings "
        "for a '{job_title}' role in '{city_location}', matching a candidate's skill set: '{formatted_skills}'. "
        "Use SerperAPI to access real-time job listings across platforms like LinkedIn, Indeed, and Google Jobs."
    ),
    backstory=("An expert job scout with years of experience navigating job boards and aggregators. "
        "This agent uses APIs and search tools to discover up-to-date roles that match the user's profile "
        "and geographic preferences. Especially skilled in using SerperAPI to find hidden job opportunities."
    ),
    tools=[serper_api_dev_tool, web_reader_tool],  # Serper api dev tool
    verbose=True
)

# Interview agent recives questions from SerperAPI
interview_agent = Agent(
    name="Interview Question Curator",
    role="AI-Powered Interview Drillmaster",
    goal=(
        "Generate a focused list of real-world interview questions asked in the past 2–3 months "
        "for the role of '{job_title}' in '{city_location}', tailored to the candidate’s skill set: "
        "'{formatted_skills}' and '{yrs_of_exp}' years of experience."
        "Divide interview questions in 3 categories DSA, System Design and skills related questions. Give a list as per the three categories listed. "
    ),
    backstory=(
        "A seasoned AI agent trained on recent interview trends and job market data. "
        "Specialized in surfacing the most frequently asked and high-impact interview questions, "
        "helping candidates simulate realistic tech interviews."
    ),
    tools=[serper_api_dev_tool, web_reader_tool],
    verbose=True
)


#Coach agent/Critic CoachAgent a personality (e.g., strict recruiter, encouraging mentor)
coach_agent = Agent(
    name="Career Critic",
    role="Blunt Career Critic",
    goal=(
        "Act as a no-nonsense career critic for a '{job_title}' with '{yrs_of_exp}' years of experience "
        "in '{', '.join(skills)}'. Analyze resume and job match data to deliver hard-hitting feedback: "
        "call out outdated or vague skills, point out resume weaknesses, recommend only high-impact learning areas, "
        "and deliver interview questions that challenge the candidate's technical depth."
    ),
    backstory=(
        "A brutally honest tech industry veteran who has helped hundreds of developers break into top-tier roles. "
        "They don’t sugarcoat anything — they critique, challenge, and demand excellence. "
        "If the resume is bloated with fluff, it gets called out. "
        "If the skills are outdated or shallow, they’ll say it plainly. "
        "Their mission: to prepare candidates to survive the toughest tech interviews and stand out as serious contenders."
    ),
    tools=[],
    verbose=True
)

#-----------Create all TASKS for each agent-------------
resume_analysis_task = Task(
    name="Analyze Resume",
    description=f"Examine the resume text including '{job_title}', '{yrs_of_exp}' and '{formatted_skills}'"
    "Extract strong skills from resume text as per market trend, identify any outdated or weak skills.",
    expected_output=(
        f"A dictionary with: Strong Skills (list), Weak Spots (list of outdated or missing skills). Exlplain why are these weak skills."
    ),
    agent=resume_analyzer_agent
)

job_search_task = Task(
    name="Fetch Relevant Job Listings",
    description=f"Search for jobs based on the '{job_title}', '{yrs_of_exp}', '{formatted_skills}', and '{city_location}' "
        "Use SERPER API to retrieve jobs with job title, company, description, and skills required and salary offered",
    expected_output=(
        "A DataFrame of job listings with : Job Title, Company, Location, Description, and Skills columns."
    ),
    agent=job_search_agent,
    context=[resume_analysis_task]  # This task uses the output of the resume_analysis_task as context
)

interview_agent_task = Task(
    name="Generate Interview Questions",
    description=(
        f"Based on the candidate's role as '{job_title}' in '{city_location}', with {yrs_of_exp} years of experience "
        f"and skills: {formatted_skills}, generate a list of the most relevant and commonly asked technical interview questions. "
        "Focus on questions that have appeared in interviews over the past 2–3 months, across job listings and hiring discussions. "
        "Prioritize questions that test practical skill depth, real-world debugging, architecture understanding, or system design."
    ),
    expected_output=(
        "A well-organized list of 5–10 recent and realistic interview questions for the specified role and skills. "
        "Questions should reflect current hiring trends, challenge depth of understanding, and help candidates prepare for high-quality technical interviews."
        "Divide interview questions in 3 categories DSA, System Design and skills related questions. Give a list as per the three categories listed. "
    ),
    agent=interview_agent,
    context=[resume_analysis_task, job_search_task]
)

coach_task = Task(
    name="Career Feedback and Coaching",
    description=f"Using the '{job_title}', '{yrs_of_exp}', '{formatted_skills}' and '{work_exp}' provide a tough, critical career review. "
        "Point out resume weaknesses, suggest 2–3 high-impact learning areas, and paraphrase the sentences/words chosen for the work expirence for better reach with recruiters",
    expected_output=(
        "A blunt but useful career feedback report, including: improvements, suggest new technology learnings with link to websites, to stay updated and relevant in job market"
        "and show any resume tone rewrites needed."
    ),
    agent=coach_agent,
    context=[resume_analysis_task]
)

# Assemble all crew agents and tasks
career_coach_crew = Crew(
    name="Smart Career Coach Crew",
    agents=[resume_analyzer_agent, job_search_agent, interview_agent, coach_agent],
    tasks=[resume_analysis_task, job_search_task, interview_agent_task, coach_task],
    process=Process.sequential,  # Executes tasks in sequence
    verbose=True
)

# Execute the crew's tasks
result = career_coach_crew.kickoff()

# Render the final refined report as markdown
display(Markdown(result.raw))

/usr/local/lib/python3.11/dist-packages/alembic/config.py:592: DeprecationWarning: No path_separator found in configuration; falling back to legacy splitting on spaces, commas, and colons for prepend_sys_path.  Consider adding path_separator=os to Alembic config.
  util.warn_deprecated(


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: Smart Career Coach Crew                                                                                  │
│  ID: 3e7ed6b8-f7e8-4a5b-be3b-a6956ac368ab                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Resume Analyser                                                                                         │
│                                                                                                                 │
│  Task: Examine the resume text including 'Software Engineer', '5' and 'Java, Python, Spring-Boot, Spring MVC,   │
│  Oracle, Cassandra, Couchbase, GIT, Grafana, Prometheus, Kafka, Microsoft Azure, Java Script'Extract strong     │
│  skills from resume text as per market trend, identify any outdated or weak skills.                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Resume Analyser                                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {                                                                                                              │
│      "Strong Skills": [                                                                                         │
│          "Java",                                                                                                │
│          "Python",                                                                                              │
│          "Spring-Boot",                                                                                         │
│          "Spring MVC",                                                                                          │
│          "GIT",                                                                                                 │
│          "Grafana",                                                                                             │
│          "Prometheus",                                                                                          │
│          "Kafka",                                                                                               │
│          "Microsoft Azure"                                                                                      │
│      ],                                                                                                         │
│      "Weak Spots": [                                                                                            │
│          "Oracle",                                                                                              │
│          "Cassandra",                                                                                           │
│          "Couchbase",                                                                                           │
│          "Java Script"                                                                                          │
│      ]                                                                                                          │
│  }                                                                                                              │
│  The strong skills listed reflect current market demand, particularly in the fields of cloud computing,         │
│  microservices, and software development best practices, such as using frameworks like Spring-Boot and Spring   │
│  MVC.                                                                                                           │
│                                                                                                                 │
│  On the other hand, the weak spots such as "Oracle", "Cassandra", "Couchbase", and "Java Script" may be         │
│  considered outdated or less relevant in certain contexts. While Oracle is still widely used, many              │
│  organizations are moving towards more flexible or open-source databases like PostgreSQL or MongoDB.            │
│  Similarly, "Cassandra" and "Couchbase" are niche technologies that aren't as broadly demanded compared to      │
│  more popular databases. "Java Script" is a relevant skill in web development; however, the trend in modern     │
│  development is leaning towards frameworks like React or Angular which integrate more seamlessly with Java      │
│  environments. Upgrading proficiency in these areas cou

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Blunt Career Critic                                                                                     │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Let's dive into your resume's reality check. You have 5 years of experience, which puts you in a decent        │
│  position. However, you need to step up your game if you want to catch the eye of recruiters at top companies.  │
│                                                                                                                 │
│  **Weaknesses:**                                                                                                │
│  1. **Vagueness in Responsibilities:** Phrases like "worked on JIRA stories" and "added APIs" don't tell me     │
│  anything about your impact or technical depth. What was the outcome? How did your contributions drive          │
│  success? You need metrics or specific challenges solved to quantify your abilities.                            │
│                                                                                                                 │
│  2. **Outdated Skill Areas:** While Oracle, Cassandra, Couchbase, and JavaScript are part of your skill set,    │
│  they aren't cutting it in today's landscape. Oracle is still used, yes, but it's becoming less of a focus      │
│  relative to more modern solutions. Similarly, Cassandra and Couchbase are niche. Upgrading to more             │
│  widely-used technologies will enhance your attractiveness.                                                     │
│                                                                                                                 │
│  3. **Buzzword Overload:** Terms like "secured Kafka data streams" need to be put into context. What was the    │
│  threat model? How did you implement OAuth 2.0? Generic statements don't resonate; specifics do.                │
│                                                                                                                 │
│  **High-Impact Learning Areas:**                                                                                │
│  1. **PostgreSQL:** Move beyond Oracle. It’s gaining traction and is used widely across industries. Check:      │
│  [PostgreSQL Official Site](https://www.postgresql.org/learn/)                                                  │
│                                                                                                                 │
│  2. **React or Angular:** Improve your JavaScript skills by learning a modern front-end framework. These        │
│  skills are essential for full-stack positions. Check: [React Official                                          │
│  Documentation](https://reactjs.org/docs/getting-started.html), [Angular Official                               │
│  Documentation](https://angular.io/docs)                                                                        │
│                                                                                                                 │
│  3. **Cloud-Native Technologies:** Explore Kubernetes and Docker to bolster your deployment skills in Azure.    │
│  These are the bread and butter of modern cloud applications. Check out [Docker Official                        │
│  Documentation](https://docs.docker.com/get-started/) and [Kubernetes Official                                  │
│  Documentation](https://kubernetes.io/docs/home/)                                                               │
│                                                        

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 1dcb69b0-72d9-49be-a91e-7dda9f84fba6                                                                     │
│  Agent: Blunt Career Critic                                                                                     │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: Smart Career Coach Crew                                                                                  │
│  ID: 3e7ed6b8-f7e8-4a5b-be3b-a6956ac368ab                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Let's dive into your resume's reality check. You have 5 years of experience, which puts you in   │
│  a decent position. However, you need to step up your game if you want to catch the eye of recruiters at top    │
│  companies.                                                                                                     │
│                                                                                                                 │
│  **Weaknesses:**                                                                                                │
│  1. **Vagueness in Responsibilities:** Phrases like "worked on JIRA stories" and "added APIs" don't tell me     │
│  anything about your impact or technical depth. What was the outcome? How did your contributions drive          │
│  success? You need metrics or specific challenges solved to quantify your abilities.                            │
│                                                                                                                 │
│  2. **Outdated Skill Areas:** While Oracle, Cassandra, Couchbase, and JavaScript are part of your skill set,    │
│  they aren't cutting it in today's landscape. Oracle is still used, yes, but it's becoming less of a focus      │
│  relative to more modern solutions. Similarly, Cassandra and Couchbase are niche. Upgrading to more             │
│  widely-used technologies will enhance your attractiveness.                                                     │
│                                                                                                                 │
│  3. **Buzzword Overload:** Terms like "secured Kafka data streams" need to be put into context. What was the    │
│  threat model? How did you implement OAuth 2.0? Generic statements don't resonate; specifics do.                │
│                                                                                                                 │
│  **High-Impact Learning Areas:**                                                                                │
│  1. **PostgreSQL:** Move beyond Oracle. It’s gaining traction and is used widely across industries. Check:      │
│  [PostgreSQL Official Site](https://www.postgresql.org/learn/)                                                  │
│                                                                                                                 │
│  2. **React or Angular:** Improve your JavaScript skills by learning a modern front-end framework. These        │
│  skills are essential for full-stack positions. Check: [React Official                                          │
│  Documentation](https://reactjs.org/docs/getting-started.html), [Angular Official                               │
│  Documentation](https://angular.io/docs)                                                                        │
│                                                                                                                 │
│  3. **Cloud-Native Technologies:** Explore Kubernetes and Docker to bolster your deployment skills in Azure.    │
│  These are the bread and butter of modern cloud applications. Check out [Docker Official                        │
│  Documentation](https://docs.docker.com/get-started/) 

Let's dive into your resume's reality check. You have 5 years of experience, which puts you in a decent position. However, you need to step up your game if you want to catch the eye of recruiters at top companies. 

**Weaknesses:**
1. **Vagueness in Responsibilities:** Phrases like "worked on JIRA stories" and "added APIs" don't tell me anything about your impact or technical depth. What was the outcome? How did your contributions drive success? You need metrics or specific challenges solved to quantify your abilities.
   
2. **Outdated Skill Areas:** While Oracle, Cassandra, Couchbase, and JavaScript are part of your skill set, they aren't cutting it in today's landscape. Oracle is still used, yes, but it's becoming less of a focus relative to more modern solutions. Similarly, Cassandra and Couchbase are niche. Upgrading to more widely-used technologies will enhance your attractiveness.

3. **Buzzword Overload:** Terms like "secured Kafka data streams" need to be put into context. What was the threat model? How did you implement OAuth 2.0? Generic statements don't resonate; specifics do.

**High-Impact Learning Areas:**
1. **PostgreSQL:** Move beyond Oracle. It’s gaining traction and is used widely across industries. Check: [PostgreSQL Official Site](https://www.postgresql.org/learn/)
   
2. **React or Angular:** Improve your JavaScript skills by learning a modern front-end framework. These skills are essential for full-stack positions. Check: [React Official Documentation](https://reactjs.org/docs/getting-started.html), [Angular Official Documentation](https://angular.io/docs)

3. **Cloud-Native Technologies:** Explore Kubernetes and Docker to bolster your deployment skills in Azure. These are the bread and butter of modern cloud applications. Check out [Docker Official Documentation](https://docs.docker.com/get-started/) and [Kubernetes Official Documentation](https://kubernetes.io/docs/home/)

**Tone Rewrites for Resume:**

1. **Original:** "Secured Kafka data streams using OAuth 2.0 authentication integrated with Microsoft Azure."
   **Rephrase:** "Implemented OAuth 2.0 for Kafka data stream security on Microsoft Azure, enhancing data protection against unauthorized access."

2. **Original:** "Adding application security for secure communication of applications."
   **Rephrase:** "Developed and integrated multiple layers of application security protocols, ensuring robust and secure inter-application communications."

3. **Original:** "Added 3 APIs with CRUD feature for a customer."
   **Rephrase:** "Designed and implemented three RESTful APIs featuring complete CRUD functionality, resulting in a 30% increase in the client's system efficiency."

4. **Original:** "Worked on JIRA stories."
   **Rephrase:** "Successfully managed and executed user stories within JIRA, driving project timelines and delivering features on schedule."

Remember, the tech landscape is evolving fast; if you're not learning, you're falling behind. Elevate your skills, clarify your impact, and revamp that resume to truly reflect your prowess. You're not just a developer; you need to show you're a problem solver. Get to work!

Hierarchical

In [ ]:
!pip install langchain-openai
from types import resolve_bases
# Import CrewAI components and tools
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool
from IPython.display import Markdown, display
from langchain_openai import ChatOpenAI
from crewai.tools import BaseTool
from pydantic import BaseModel, Field
# Import ChatOpenAI to serve as the manager LLM for the hierarchical process
from langchain.chat_models import ChatOpenAI

#-------User input/Resume text------------
job_title = "Software Engineer"
yrs_of_exp = "5"
skills = ["Java", "Python", "Spring-Boot", "Spring MVC", "Oracle",
        "Cassandra", "Couchbase", "GIT", "Grafana", "Prometheus",
        "Kafka", "Microsoft Azure", "Java Script"]
work_exp=["Secured Kafka data streams using OAuth 2.0 authentication integrated with MicrosoE Azure.",
          "Adding application security for secure communication of applications.",
          "Added 3 APIs with CRUD feature for a customer",
          "Worked on JIRA stories"]
city_location = "SFO"

#-------FORMAT lists from user input------------------------
# Format skills and work exp for inclusion in goal strings
formatted_skills = ", ".join(skills)  # e.g., "Java, Python, Spring-Boot, ..."
formatted_work_exp = "\n".join(work_exp)


#---------- Enables web searching via the Serper API AND Retrieves and reads content from web pages------
serper_api_dev_tool = SerperDevTool()
web_reader_tool = WebsiteSearchTool()  # Retrieves and reads content from web pages



#--------Create all AGENTS and describe its role goal and tools used -------------------------------
# Resume Analyser agent goal is to analyse resume extract key skills, exp summary, and outdated tech
resume_analyzer_agent = Agent(
    name="Analyser",
    role="Resume Analyser",
    goal=("Extract job title '{job_title}', '{yrs_of_exp}' years of experience, skills '{formatted_skills}' and work experience '{formatted_work_exp}' "
    "Identify the job title, outdated or weak skills from the list of extracted skills, provided in the resume text."
    "Do not give text which is not present in the '{formatted_skills}' and '{formatted_work_exp}'."
    "Summarize it in a short paragraph."
    ),
    backstory=("A senior career analyst specializing in resume mining. With years of expertise in technical hiring, the agent detects strengths, blind spots, "
        "and recommends learning areas based on skill trends."
    ),
    tools=[] #not required for this agent
    )

# Job Search agent retrieves relevant jobs from  SerperAPI
job_search_agent = Agent(
    name="Job Finder",
    role="Market Research and Job Search Agent",
    goal=("Search and retrieve the most relevant and high-quality job listings "
        "for a '{job_title}' role in '{city_location}', matching a candidate's skill set: '{formatted_skills}'. "
        "Use SerperAPI to access real-time job listings across platforms like LinkedIn, Indeed, and Google Jobs."
    ),
    backstory=("An expert job scout with years of experience navigating job boards and aggregators. "
        "This agent uses APIs and search tools to discover up-to-date roles that match the user's profile "
        "and geographic preferences. Especially skilled in using SerperAPI to find hidden job opportunities."
    ),
    tools=[serper_api_dev_tool],  # Serper api dev tool
    verbose=True
)

# Interview agent recives questions from SerperAPI
interview_agent = Agent(
    name="Interview Question Curator",
    role="AI-Powered Interview Drillmaster",
    goal=(
        "Generate a focused list of real-world interview questions asked in the past 2–3 months "
        "for the role of '{job_title}' in '{city_location}', tailored to the candidate’s skill set: "
        "'{formatted_skills}' and '{yrs_of_exp}' years of experience."
        "Divide interview questions in 3 categories DSA, System Design and skills related questions. Give a list as per the three categories listed. "
    ),
    backstory=(
        "A seasoned AI agent trained on recent interview trends and job market data. "
        "Specialized in surfacing the most frequently asked and high-impact interview questions, "
        "helping candidates simulate realistic tech interviews."
    ),
    tools=[serper_api_dev_tool],
    verbose=True
)

#Coach agent/Critic CoachAgent a personality (e.g., strict recruiter, encouraging mentor)
coach_agent = Agent(
    name="Career Critic",
    role="Blunt Career Critic",
    goal=(
        "Act as a no-nonsense career critic for a '{job_title}' with '{yrs_of_exp}' years of experience "
        "in '{', '.join(skills)}'. Analyze resume and job match data to deliver hard-hitting feedback: "
        "call out outdated or vague skills, point out resume weaknesses, recommend only high-impact learning areas, "
        "and deliver interview questions that challenge the candidate's technical depth."
    ),
    backstory=(
        "A brutally honest tech industry veteran who has helped hundreds of developers break into top-tier roles. "
        "They don’t sugarcoat anything — they critique, challenge, and demand excellence. "
        "If the resume is bloated with fluff, it gets called out. "
        "If the skills are outdated or shallow, they’ll say it plainly. "
        "Their mission: to prepare candidates to survive the toughest tech interviews and stand out as serious contenders."
    ),
    tools=[],
    verbose=True
)

#--------Define TASKS for each agent-------------

resume_analysis_task = Task(
    name="Analyze Resume",
    description=f"Examine the resume text including '{job_title}', '{yrs_of_exp}' and '{formatted_skills}'"
    "Extract strong skills from resume text as per market trend, identify any outdated or weak skills.",
    expected_output=(
        f"A dictionary with: Strong Skills (list), Weak Spots (list of outdated or missing skills). Exlplain why are these weak skills."
    ),
    agent=resume_analyzer_agent
)

job_search_task = Task(
    name="Fetch Relevant Job Listings",
    description=f"Search for jobs based on the '{job_title}', '{yrs_of_exp}', '{formatted_skills}', and '{city_location}' "
        "Use SERPER API to retrieve jobs with job title, company, description, and skills required and salary offered",
    expected_output=(
        "A DataFrame of job listings with : Job Title, Company, Location, Description, and Skills columns."
    ),
    agent=job_search_agent,
    context=[resume_analysis_task]  # This task uses the output of the resume_analysis_task as context
)

interview_agent_task = Task(
    name="Generate Interview Questions",
    description=(
        f"Based on the candidate's role as '{job_title}' in '{city_location}', with {yrs_of_exp} years of experience "
        f"and skills: {formatted_skills}, generate a list of the most relevant and commonly asked technical interview questions. "
        "Focus on questions that have appeared in interviews over the past 2–3 months, across job listings and hiring discussions. "
        "Prioritize questions that test practical skill depth, real-world debugging, architecture understanding, or system design."
    ),
    expected_output=(
        "A well-organized list of 5–10 recent and realistic interview questions for the specified role and skills. "
        "Questions should reflect current hiring trends, challenge depth of understanding, and help candidates prepare for high-quality technical interviews."
        "Divide interview questions in 3 categories DSA, System Design and skills related questions. Give a list as per the three categories listed. "
    ),
    agent=interview_agent,
    context=[resume_analysis_task, job_search_task]
)

coach_task = Task(
    name="Career Feedback and Coaching",
    description=f"Using the '{job_title}', '{yrs_of_exp}', '{formatted_skills}' and '{work_exp}' provide a tough, critical career review. "
        "Point out resume weaknesses, suggest 2–3 high-impact learning areas, and paraphrase the sentences/words chosen for the work expirence for better reach with recruiters",
    expected_output=(
        "A blunt but useful career feedback report, including: improvements, suggest new technology learnings with link to websites, to stay updated and relevant in job market"
        "and show any resume tone rewrites needed."
    ),
    agent=coach_agent,
    context=[resume_analysis_task]
)


# ---------Set up the LLM-based manager for the hierarchical process---------
manager_llm = ChatOpenAI(temperature=0, model="gpt-4")

# ---------Assemble all crew agents and tasks----------------
career_coach_crew = Crew(
    name="Smart Career Coach Crew",
    agents=[resume_analyzer_agent, job_search_agent, interview_agent, coach_agent],
    tasks=[resume_analysis_task, job_search_task, interview_agent_task, coach_task],
    process=Process.hierarchical,  # Use the hierarchical process
    manager_llm=manager_llm,       # Provide the LLM-based manager required for hierarchical execution
    verbose=True
)

# --------Execute the crew's tasks--------------
result = career_coach_crew.kickoff()

#--------Render the final refined report as markdown-----
display(Markdown(result.raw))


🚀 Crew: Smart Career Coach Crew
├── 📋 Task: c1f81b6a-9430-44b4-aa31-6cce889c7919
│   Assigned to: Crew Manager
│   Status: ✅ Completed
│   └── 🔧 Using Delegate work to coworker (4)
├── 📋 Task: 1608d6e7-0242-4b8b-9b3b-7fcaf7cff7fa
│   Assigned to: Crew Manager
│   Status: ✅ Completed
│   ├── 🔧 Used Delegate work to coworker (5)
│   ├── 🔧 Using Delegate work to coworker (6)
│   └── 🔧 Used Search the internet with Serper (5)
├── 📋 Task: 034e7265-2c96-49b1-ad70-31b61b37e77c
│   Assigned to: Crew Manager
│   Status: ✅ Completed
│   ├── 🔧 Used Search the internet with Serper (6)
│   ├── 🔧 Using Delegate work to coworker (7)
│   └── 🔧 Used Search the internet with Serper (7)
└── 📋 Task: 7fd8e3d4-42cd-4127-a42e-411282346989
    Status: Executing Task...
    └── 🔧 Using Delegate work to coworker (8)

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Crew Manager                                                                                            │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Let's break this down critically.                                                                              │
│                                                                                                                 │
│  **Resume Weaknesses:**                                                                                         │
│  1. **Outdated Skills:** You list 'Oracle' and 'JavaScript' as weak points. In the current tech landscape, if   │
│  you're not actively applying these skills or leveraging modern frameworks (like React or Angular for           │
│  JavaScript), it's time to either sharpen those skills or expand your repertoire with tools that are more in    │
│  demand.                                                                                                        │
│                                                                                                                 │
│  2. **Vague Descriptions:** If your work experience highlights duties rather than achievements, it’s not doing  │
│  you any favors. Recruiters want to see quantifiable impacts. Avoid phrases like “involved in projects” and     │
│  instead say “led a project that improved system performance by 30%”.                                           │
│                                                                                                                 │
│  3. **Lack of Focus on Cloud Technologies:** You have 'Microsoft Azure' in your skill set, which is a plus.     │
│  However, without concrete examples of how you've utilized it, it may lose significance. Make sure to detail    │
│  specific projects or applications you’ve developed in Azure.                                                   │
│                                                                                                                 │
│  **High-Impact Learning Areas:**                                                                                │
│  1. **Modern JavaScript Frameworks:** Level up with frameworks such as React or Vue.js. Not only will this      │
│  bolster your resume, but it will also position you as a more versatile developer capable of contributing to    │
│  full-stack projects.                                                                                           │
│                                                                                                                 │
│  2. **Cloud-Native Databases:** Since you have experience with Azure, dive deeper into Azure services or        │
│  expand your knowledge to other cloud-native databases (like Cosmos DB or AWS DynamoDB). Understanding          │
│  serverless architecture could also give you a competitive edge.                                                │
│                                                                                                                 │
│  3. **Microservices Architecture:** Strengthen your understanding of microservices and containerization         │
│  technologies like Docker and Kubernetes. These are essential in modern software development and can set you    │
│  apart in tech interviews.                                                                                      │
│                                                                                                                 │
│  **Paraphrasing Work Experience:**                     

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 7fd8e3d4-42cd-4127-a42e-411282346989                                                                     │
│  Agent: Crew Manager                                                                                            │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: Smart Career Coach Crew                                                                                  │
│  ID: 1852d753-c6df-4564-80e1-aaaab9e914fc                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Let's break this down critically.                                                                │
│                                                                                                                 │
│  **Resume Weaknesses:**                                                                                         │
│  1. **Outdated Skills:** You list 'Oracle' and 'JavaScript' as weak points. In the current tech landscape, if   │
│  you're not actively applying these skills or leveraging modern frameworks (like React or Angular for           │
│  JavaScript), it's time to either sharpen those skills or expand your repertoire with tools that are more in    │
│  demand.                                                                                                        │
│                                                                                                                 │
│  2. **Vague Descriptions:** If your work experience highlights duties rather than achievements, it’s not doing  │
│  you any favors. Recruiters want to see quantifiable impacts. Avoid phrases like “involved in projects” and     │
│  instead say “led a project that improved system performance by 30%”.                                           │
│                                                                                                                 │
│  3. **Lack of Focus on Cloud Technologies:** You have 'Microsoft Azure' in your skill set, which is a plus.     │
│  However, without concrete examples of how you've utilized it, it may lose significance. Make sure to detail    │
│  specific projects or applications you’ve developed in Azure.                                                   │
│                                                                                                                 │
│  **High-Impact Learning Areas:**                                                                                │
│  1. **Modern JavaScript Frameworks:** Level up with frameworks such as React or Vue.js. Not only will this      │
│  bolster your resume, but it will also position you as a more versatile developer capable of contributing to    │
│  full-stack projects.                                                                                           │
│                                                                                                                 │
│  2. **Cloud-Native Databases:** Since you have experience with Azure, dive deeper into Azure services or        │
│  expand your knowledge to other cloud-native databases (like Cosmos DB or AWS DynamoDB). Understanding          │
│  serverless architecture could also give you a competitive edge.                                                │
│                                                                                                                 │
│  3. **Microservices Architecture:** Strengthen your understanding of microservices and containerization         │
│  technologies like Docker and Kubernetes. These are essential in modern software development and can set you    │
│  apart in tech interviews.                                                                                      │
│                                                       

Let's break this down critically.

**Resume Weaknesses:**
1. **Outdated Skills:** You list 'Oracle' and 'JavaScript' as weak points. In the current tech landscape, if you're not actively applying these skills or leveraging modern frameworks (like React or Angular for JavaScript), it's time to either sharpen those skills or expand your repertoire with tools that are more in demand.
   
2. **Vague Descriptions:** If your work experience highlights duties rather than achievements, it’s not doing you any favors. Recruiters want to see quantifiable impacts. Avoid phrases like “involved in projects” and instead say “led a project that improved system performance by 30%”.

3. **Lack of Focus on Cloud Technologies:** You have 'Microsoft Azure' in your skill set, which is a plus. However, without concrete examples of how you've utilized it, it may lose significance. Make sure to detail specific projects or applications you’ve developed in Azure.

**High-Impact Learning Areas:**
1. **Modern JavaScript Frameworks:** Level up with frameworks such as React or Vue.js. Not only will this bolster your resume, but it will also position you as a more versatile developer capable of contributing to full-stack projects.
   
2. **Cloud-Native Databases:** Since you have experience with Azure, dive deeper into Azure services or expand your knowledge to other cloud-native databases (like Cosmos DB or AWS DynamoDB). Understanding serverless architecture could also give you a competitive edge.

3. **Microservices Architecture:** Strengthen your understanding of microservices and containerization technologies like Docker and Kubernetes. These are essential in modern software development and can set you apart in tech interviews.

**Paraphrasing Work Experience:**
Instead of saying “Worked on building Java applications,” consider: “Engineered scalable Java applications that processed over 1 million transactions daily, enhancing system reliability and user experience.”

Instead of “Used Spring-Boot framework,” try: “Leveraged the Spring-Boot framework to develop RESTful services that reduced development time by 25% and streamlined API integration.”

Moving forward, structure your resume to focus on achievements rather than tasks. Demonstrate clear metrics and outcomes. This will not only appeal to recruiters but also prepare you for rigorous technical interviews where you will need to articulate your contributions in detail.

Prepare for interview questions like:
- "Can you explain the trade-offs of using microservices versus a monolithic architecture in your previous projects?"
- "Describe a challenging bug you encountered with Kafka messaging and how you resolved it."
- "How have you integrated monitoring tools like Grafana and Prometheus into a microservices architecture?" 

By sharpening your focus on your strengths, cutting out outdated skills, and showcasing powerful experiences, you'll elevate your candidacy significantly. Good luck!